<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/1_crawling/c3_crawling_mod_comp_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

# 예외 케이스 탐색

In [1]:
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

## local

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

# driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe', options=options)
driver = webdriver.Chrome()

# driver.get(url)

In [54]:
# driver.window_handles

# driver.switch_to.window(driver.window_handles[-1])
# driver.switch_to.frame("lawViewTitle")

# for d in driver.find_elements(By.CSS_SELECTOR, "html > body > div > div"):
#     print(d.get_attribute("class"))

In [2]:
def wait_driver(xpath, t=10):
    WebDriverWait(driver, t).until(EC.presence_of_element_located((By.XPATH, xpath)))
    
def tab_move():
    driver.switch_to.window(driver.window_handles[-1])
    ko_url = driver.current_url.replace("ENG", "KOR")
    # driver.get(ko_url)
    wait_driver('//*[@id="lawViewTitle"]')
    # driver.switch_to.frame("lawViewContent")

def tab_close():
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("indexFrame")

def parsing(cate, law):
    contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')

    eng_line, kor_line = [], []
    for i in range(0, len(contents), 2):
        cur_eng, cur_kor = [], []
        eng_idx, kor_idx = i, i+1
        if contents[eng_idx].text=='' or contents[kor_idx].text=='':
            # print(contents[eng_idx].text)
            continue
        for key in ["none", "ho", "hang"]:
            cur_eng = list(map(lambda x: x.text, contents[eng_idx].find_elements(By.CLASS_NAME, key)))
            cur_kor = list(map(lambda x: x.text, contents[kor_idx].find_elements(By.CLASS_NAME, key)))
            if len(cur_eng)==len(cur_kor):
                eng_line+=cur_eng
                kor_line+=cur_kor
            else:
                return law
    return pd.DataFrame(zip([cate]*len(eng_line), [law]*len(kor_line), eng_line, kor_line), columns=["category", "lawName", "ENG", "KOR"])

In [3]:
## crawler total

driver.get(url)
categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
# exception_list = []
jc_list, non_jc_list = [], []
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    # if j<18: continue
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    df_laws = pd.DataFrame()
    cate_jc_list=[]
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        page_exception_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            
            # JC 검색
            driver.switch_to.frame("lawViewTitle")
            ltv = driver.find_elements(By.CSS_SELECTOR, "html > body > div > div")
            # print(f"=====LAW NAME : {law_name}========")
            for id in ltv:
                id_name = id.get_attribute('class')
                # print(id_name)
                if id_name=="lawnametitle": continue
                elif id_name=="JC2":
                    cate_jc_list.append(law_name)
                    break

            # fininsh parsing
            tab_close()
            # if type(result)==str:
            #     exception_list.append(result)
            #     page_exception_num+=1
            # else:
            #     df_laws = pd.concat([df_laws, result], axis=0)
            #     page_parsing_num+=1
            page_parsing_num+=1
        # print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}\t예외처리: {page_exception_num}")
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}\t누적 예외처리: {len(cate_jc_list)}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    jc_list+=cate_jc_list    
    print(f"{cate_name} 파싱 완료\t({len(cate_jc_list)})\n------------------------------------------")
    # df_laws.to_csv(save_path+f"/{cate_name}.csv", index=False, encoding='euc-kr')
    # base_df = pd.concat([base_df, df_laws], axis=0)
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()

1. 헌법 시작...
	1페이지 파싱 완료: 20	누적 예외처리: 1
	2페이지 파싱 완료: 11	누적 예외처리: 2
헌법 파싱 완료	(2)
------------------------------------------
2. 국회 시작...
	1페이지 파싱 완료: 10	누적 예외처리: 1
국회 파싱 완료	(1)
------------------------------------------
3. 선거ㆍ정당 시작...
	1페이지 파싱 완료: 8	누적 예외처리: 0
선거ㆍ정당 파싱 완료	(0)
------------------------------------------
4. 행정일반 시작...
	1페이지 파싱 완료: 20	누적 예외처리: 0
	2페이지 파싱 완료: 20	누적 예외처리: 0
	3페이지 파싱 완료: 9	누적 예외처리: 0
행정일반 파싱 완료	(0)
------------------------------------------
5. 국가공무원 시작...
	1페이지 파싱 완료: 20	누적 예외처리: 0
	2페이지 파싱 완료: 14	누적 예외처리: 0
국가공무원 파싱 완료	(0)
------------------------------------------
6. 법원 시작...
	1페이지 파싱 완료: 8	누적 예외처리: 0
법원 파싱 완료	(0)
------------------------------------------
7. 법무 시작...
	1페이지 파싱 완료: 20	누적 예외처리: 0
	2페이지 파싱 완료: 20	누적 예외처리: 0
	3페이지 파싱 완료: 1	누적 예외처리: 0
법무 파싱 완료	(0)
------------------------------------------
8. 민사법 시작...
	1페이지 파싱 완료: 20	누적 예외처리: 0
	2페이지 파싱 완료: 20	누적 예외처리: 1
	3페이지 파싱 완료: 20	누적 예외처리: 1
	4페이지 파싱 완료: 20	누적 예외처리: 2
	5페이지 파싱 완료: 17	누적 예외처리: 2
민사법 파싱 완료	(2)

In [4]:
print(len(jc_list))
jc_list

27


['대한민국헌법',
 '연호에 관한 법률',
 '국회법',
 '민법',
 '외국인의 서명날인에 관한 법률',
 '비영리법인의 임원 처벌에 관한 법률',
 '표준시에 관한 법률',
 '경품류제공에관한불공정거래행위의유형및기준지정고시',
 '과징금부과 세부기준 등에 관한 고시',
 '기술자료 제공 요구·유용행위 심사지침',
 '기업결합 심사기준',
 '법령 등의 경쟁제한사항 심사지침',
 '부당특약 심사지침',
 '용역위탁 중 역무의 범위 고시',
 '중요한 표시·광고사항 고시',
 '표시·광고의 공정화에 관한 법률 위반사업자등에 대한 과징금부과 세부기준 등에 관한 고시',
 '하도급거래공정화지침',
 '특정국가와의 관세협상에 따른 국제협력관세의 적용에 관한 규정',
 '대규모 유통업 분야의 특약매입거래에 관한 부당성 심사지침',
 '벤처투자 촉진에 관한 법률 시행규칙',
 '선불식 할부거래에서의 소비자보호 지침',
 '전자상거래 등에서의 소비자보호 지침',
 '전자상거래 소비자보호법 위반사업자에 대한 과징금 부과기준 고시',
 '국가핵심기술',
 '근로자의 날 제정에 관한 법률',
 '선원법 시행규칙',
 '선원의안전및위생에관한규칙']

In [6]:
import pickle

with open("article_cont.pickle", "wb") as f:
    pickle.dump(jc_list, f)

In [9]:
with open("article_cont.pickle", "rb") as f:
    jc = pickle.load(f)
jc

['대한민국헌법',
 '연호에 관한 법률',
 '국회법',
 '민법',
 '외국인의 서명날인에 관한 법률',
 '비영리법인의 임원 처벌에 관한 법률',
 '표준시에 관한 법률',
 '경품류제공에관한불공정거래행위의유형및기준지정고시',
 '과징금부과 세부기준 등에 관한 고시',
 '기술자료 제공 요구·유용행위 심사지침',
 '기업결합 심사기준',
 '법령 등의 경쟁제한사항 심사지침',
 '부당특약 심사지침',
 '용역위탁 중 역무의 범위 고시',
 '중요한 표시·광고사항 고시',
 '표시·광고의 공정화에 관한 법률 위반사업자등에 대한 과징금부과 세부기준 등에 관한 고시',
 '하도급거래공정화지침',
 '특정국가와의 관세협상에 따른 국제협력관세의 적용에 관한 규정',
 '대규모 유통업 분야의 특약매입거래에 관한 부당성 심사지침',
 '벤처투자 촉진에 관한 법률 시행규칙',
 '선불식 할부거래에서의 소비자보호 지침',
 '전자상거래 등에서의 소비자보호 지침',
 '전자상거래 소비자보호법 위반사업자에 대한 과징금 부과기준 고시',
 '국가핵심기술',
 '근로자의 날 제정에 관한 법률',
 '선원법 시행규칙',
 '선원의안전및위생에관한규칙']

In [ ]:
## crawler total

driver.get(url)
categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
# exception_list = []
total_zero_list = []
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    # if j<18: continue
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    df_laws = pd.DataFrame()
    zero_list = []
    while True:
    # laws loop
        page_zero_list=[]
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        page_exception_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            
            # JC 검색
            driver.switch_to.frame("lawViewTitle")
            ltv = driver.find_elements(By.CSS_SELECTOR, "html > body > div > div")[1:]
            # print(f"=====LAW NAME : {law_name}========")
            if len(ltv)==0:
                page_zero_list.append(law_name)

            # fininsh parsing
            tab_close()
            # if type(result)==str:
            #     exception_list.append(result)
            #     page_exception_num+=1
            # else:
            #     df_laws = pd.concat([df_laws, result], axis=0)
            #     page_parsing_num+=1
            page_parsing_num+=1
        # print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}\t예외처리: {page_exception_num}")
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}\t페이지 예외처리: {len(page_zero_list)}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
            zero_list+=page_zero_list
        else:
            zero_list+=page_zero_list
            break
    total_zero_list+=zero_list    
    print(f"{cate_name} 파싱 완료\t({len(zero_list)})\t(total-{len(total_zero_list)})\n------------------------------------------")
    # df_laws.to_csv(save_path+f"/{cate_name}.csv", index=False, encoding='euc-kr')
    # base_df = pd.concat([base_df, df_laws], axis=0)
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()

In [13]:
total_zero_list

['인지 첩부·첨부 및 공탁 제공에 관한 특례법']

In [10]:
non_article_conts = ['인지 첩부·첨부 및 공탁 제공에 관한 특례법']
with open('article_contents/non_article_conts.pickle', 'wb') as f:
    pickle.dump(non_article_conts, f, protocol=pickle.HIGHEST_PROTOCOL)

# 이전 예외 케이스와 비교

In [5]:
import pickle

with open(r"article_contents\article_cont.pickle", "rb") as f:
    jc2_list = pickle.load(f)

with open(r"article_contents\non_article_conts.pickle", "rb") as f:
    non_art_list = pickle.load(f)
    
try:
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
    with open(gdrive_path+r"\etc_files\exception_list.pkl", "rb") as f:
        exception_list = pickle.load(f)
except:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"
    with open(gdrive_path+r"\etc_files\exception_list.pkl", "rb") as f:
        exception_list = pickle.load(f)

In [6]:
extra_except_list = jc2_list + non_art_list

print(f"크롤링 시 제외: {len(exception_list)}\tJC2 class 제외: {len(jc2_list)}\t목차 태그 없음: {len(non_art_list)}")
print(f"원래 제외 항목 - 추가 제외 항목: {len(list(set(exception_list)-set(extra_except_list)))}")
print(f"추가 제외 항목-원래 제외 항목: {len(list(set(extra_except_list)-set(exception_list)))}")

크롤링 시 제외: 1193	JC2 class 제외: 27	목차 태그 없음: 1
원래 제외 항목 - 추가 제외 항목: 1176
추가 제외 항목-원래 제외 항목: 11


In [7]:
add_collect = list(set(exception_list)-set(extra_except_list))
with open(r"article_contents\add_collect.pickle", "wb") as f:
    pickle.dump(add_collect, f)

# 수집된 Raw 데이터에서 보기

In [6]:
import pandas as pd

df_raw = pd.read_csv(gdrive_path+"/export_crawl_1/crawl_total.csv", encoding="euc-kr")
df_raw.head()

,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발ㆍ운영의 지원 및 개성공업지구에 투자하거나 출입ㆍ체류하는...
1,헌법,개성공업지구 지원에 관한 법률,The terms used in this Act are defined as foll...,"이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2009. 1. 30., 2..."
2,헌법,개성공업지구 지원에 관한 법률,1. The term “Gaeseong Industrial Complex” or “...,1. “개성공업지구”란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 ...
3,헌법,개성공업지구 지원에 관한 법률,2. The term “GIC developer” means a South Kore...,2. “개성공업지구 개발업자”란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인...
4,헌법,개성공업지구 지원에 관한 법률,3. The term “GIC management organ” means a cor...,3. “개성공업지구 관리기관”이란 개성공업지구의 관리ㆍ운영을 위하여 북한의 「개성공...


In [ ]:
# 수집 됐지만, JC2 Tag 이거나 목차태그가 없는 경우

case1 = list(set(extra_except_list)-set(exception_list))
df_c1 = pd.DataFrame()
for ln in case1:
    df_c1 = pd.concat([df_c1, df_raw.loc[df_raw.lawName==ln]], axis=0).reset_index(drop=True)
df_c1

,category,lawName,ENG,KOR
0,국회,국회법,The purpose of this Act is to contribute to th...,"이 법은 국회의 조직ㆍ의사(議事), 그 밖에 필요한 사항을 규정함으로써 국민의 대의..."
1,국회,국회법,"[This Article Wholly Amended on Apr. 17, 2018]",[전문개정 2018. 4. 17.]
2,국회,국회법,"[This Article Wholly Amended on Apr. 17, 2018]",[전문개정 2018. 4. 17.]
3,국회,국회법,(1) When those who are elected as members of t...,① 중앙선거관리위원회 위원장은 국회의원 당선인이 결정된 때에는 그 명단을 즉시 국회...
4,국회,국회법,(2) Those who are elected as members of the Na...,② 국회의원 당선인은 당선인으로 결정된 후 당선증서를 국회사무처에 제시하고 등록하여...
...,...,...,...,...
1111,헌법,연호에 관한 법률,(1) This Act shall enter into force on January...,① 본법은 단기 4295년 1월 1일부터 시행한다.
1112,헌법,연호에 관한 법률,(2) The Reign-Title Act (Act No. 4) is hereby ...,② 법률 제4호 연호에관한법률은 이를 폐지한다.
1113,헌법,연호에 관한 법률,(3) Any year indicated in the Dangun Era in of...,③ 본법 시행당시의 공문서중 단기로 표시된 연대는 당해 단기연대에서 2333년을 감...
1114,헌법,연호에 관한 법률,(4) Any year may be corrected by affixing a co...,④ 연대 정정에 있어서는 공문서정정에 관한 타 법령의 규정에 불구하고 당해 공문서의...


In [ ]:
df_raw = df_raw.dropna(subset=["KOR"], axis=0).reset_index(drop=True)

In [31]:
df_raw.loc[df_raw.ENG.str.contains("^\([\d]+\)")]["ENG"].apply(lambda x: int(x.split(" ")[0].strip("()"))).sort_values(ascending=False)

153178    15
171819    15
72667     14
149187    14
171818    14
          ..
38828      1
142447     1
38826      1
142450     1
9          1
Name: ENG, Length: 94311, dtype: int64

# 예외 케이스 탐색 시뮬레이션

In [8]:
import pickle

with open("article_contents/add_collect.pickle", "rb") as f:
    add_collect = pickle.load(f)

In [12]:
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
driver = webdriver.Chrome()

In [13]:
driver.get(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "indexFrame")))
driver.switch_to.frame("indexFrame")

In [22]:
def wait_driver(xpath, t=10):
    WebDriverWait(driver, t).until(EC.presence_of_element_located((By.XPATH, xpath)))
    
def tab_move():
    driver.switch_to.window(driver.window_handles[-1])
    ko_url = driver.current_url.replace("ENG", "KOR")
    driver.get(ko_url)
    wait_driver('//*[@id="lawViewTitle"]')
    driver.switch_to.frame("lawViewContent")

def tab_close():
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("indexFrame")

def parsing(cate, law):
    contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')

    eng_line, kor_line = [], []
    eng_expt, kor_expt = [], []
    for i in range(0, len(contents), 2):
        cur_eng, cur_kor = [], []
        eng_idx, kor_idx = i, i+1
        if contents[eng_idx].text=='' or contents[kor_idx].text=='':
            # print(contents[eng_idx].text)
            continue
        for key in ["none", "ho", "hang"]:
            cur_eng = list(map(lambda x: x.text, contents[eng_idx].find_elements(By.CLASS_NAME, key)))
            cur_kor = list(map(lambda x: x.text, contents[kor_idx].find_elements(By.CLASS_NAME, key)))
            if len(cur_eng)==len(cur_kor):
                eng_line+=cur_eng
                kor_line+=cur_kor
            else:
                eng_len, kor_len = len(cur_eng), len(cur_kor)
                if eng_len > kor_len:
                    cur_kor+=['']*(eng_len-kor_len)
                else:
                    cur_eng+=['']*(kor_len*eng_len)
                eng_expt+=cur_eng
                kor_expt+=cur_kor
                continue
    df_laws = pd.DataFrame(zip([cate]*len(eng_line), [law]*len(kor_line), eng_line, kor_line), columns=["category", "lawName", "ENG", "KOR"])
    df_expt = pd.DataFrame(zip([cate]*len(eng_line), [law]*len(kor_line), eng_expt, kor_expt), columns=["category", "lawName", "ENG", "KOR"])
    return df_laws, df_expt

In [18]:
temp = pd.DataFrame(columns=['a','b','c','d'])
print(temp.shape)

(0, 4)


In [ ]:
## crawler total
try:
    driver.quit()
except:
    pass

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

driver=webdriver.Chrome(options=options)

driver.get(url)
categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
expt_df = pd.DataFrame()
base_df = pd.DataFrame()
page_exception_num = 0
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    df_laws = pd.DataFrame()
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            if law_name in add_collect:
                law.send_keys(Keys.ENTER)
                tab_move()
                wait_driver('/html')

                # Parsing
                result, expt = parsing(cate_name, law_name)
                # fininsh parsing
                tab_close()
                
                if expt.shape[0] != 0:
                    expt_df = pd.concat([expt_df, expt], axis=0).reset_index(drop=True)
                    page_exception_num+=1

                df_laws = pd.concat([df_laws, result], axis=0)
                page_parsing_num+=1
            else:
                pass
            
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}\t현재까지 예외처리: {page_exception_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    print(f"{cate_name} 파싱 완료\n------------------------------------------{df_laws.shape[0]}")
    # df_laws.to_csv(save_path+f"/{cate_name}.csv", index=False, encoding='euc-kr')
    base_df = pd.concat([base_df, df_laws], axis=0)
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()

In [24]:
expt_df

,category,lawName,ENG,KOR
0,헌법,겨레말큰사전남북공동편찬사업회법,"[This Article Newly Inserted on Jan. 15, 2019]",통일부장관은 편찬사업회의 운영이 다음 각 호의 어느 하나에 해당하는 경우에는 기간을...
1,헌법,겨레말큰사전남북공동편찬사업회법,[Previous Article 21 moved to Article 24 <Jan....,[본조신설 2019. 1. 15.]
2,헌법,겨레말큰사전남북공동편찬사업회법,,[종전 제21조는 제24조로 이동 <2019. 1. 15.>]
3,헌법,겨레말큰사전남북공동편찬사업회법,,
4,헌법,법령 등 공포에 관한 법률,The date of promulgation or public announcemen...,
...,...,...,...,...
222,행정일반,법제업무 운영규정,,[제27조의8에서 이동 <2017.5.8.>]
223,행정일반,법제업무 운영규정,,[전문개정 2010.10.5.]
224,행정일반,법제업무 운영규정,Members and related persons who have attended ...,[제목개정 2017.5.8.]
225,행정일반,온실가스 배출권의 할당 및 거래에 관한 법률,A person who is not a public official among th...,인증위원회의 위원 중 공무원이 아닌 사람은 「형법」 제129조부터 제132조까지의 ...


In [24]:
base_df.to_csv(r"G:\내 드라이브\KB인턴관련\crawl_except.csv", index=False, encoding='euc-kr')

In [26]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365455 entries, 0 to 82
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   category  365455 non-null  object
 1   lawName   365455 non-null  object
 2   ENG       365455 non-null  object
 3   KOR       365455 non-null  object
dtypes: object(4)
memory usage: 13.9+ MB


In [2]:
# len(exception_dict.keys())

In [ ]:
import time

for law in add_collect:
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "indexFrame")))
    driver.switch_to.frame("indexFrame")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="totalSearchValue"]')))
    search_bar = driver.find_element(By.XPATH, '//*[@id="totalSearchValue"]')
    search_bar.send_keys(law)
    button = driver.find_element(By.CLASS_NAME, "searching")
    button.send_keys(Keys.ENTER)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="1"]/td[2]/p/a')))
    driver.find_element(By.XPATH, '//*[@id="1"]/td[2]/p/a').click()
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(driver.current_url.replace("ENG", "KOR"))
    parsing()
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#')))
    # driver.find_element(By.XPATH, '//*[@id="lawSearch_container"]/div/div[4]/div[1]/div/div[1]/label[2]').click()
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    print(law)


# 예외 수집 데이터 확인

In [1]:
import pandas as pd

df_exe = pd.read_csv(r"G:\내 드라이브\KB인턴관련\crawl_except.csv", encoding='euc-kr')
df_exe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365455 entries, 0 to 365454
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   category  365455 non-null  object
 1   lawName   365455 non-null  object
 2   ENG       365455 non-null  object
 3   KOR       365432 non-null  object
dtypes: object(4)
memory usage: 11.2+ MB


In [2]:
df_ex = df_exe.dropna(subset=["KOR"], axis=0).reset_index(drop=True)

In [3]:
df_ex.loc[df_ex.ENG.str.contains("Omitted")]["KOR"].value_counts()[:29]

② 생략                                                                                                              407
② 및 ③ 생략                                                                                                           33
③ 생략                                                                                                               32
④ 생략                                                                                                               19
① (시행일) 이 법은 공포한 날부터 시행한다.                                                                                         12
① 이 영은 2020년 8월 5일부터 시행한다. <단서 생략>                                                                                 12
② 부터 ④까지 생략                                                                                                         9
② 및 ③생략                                                                                                             9
② 내지 ④생략                                                

In [4]:
df_ex.loc[df_ex.ENG.str.contains("Deleted")]

,category,lawName,ENG,KOR
264,헌법,남북교류협력에 관한 법률,"(3) Deleted. <May 28, 2009>",③ 삭제 <2009. 5. 28.>
494,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률,"4. Deleted; <Dec. 8, 2020>",4. 삭제 <2020. 12. 8.>
563,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률,"(5) Deleted. <Jan. 15, 2019>",⑤ 삭제 <2019. 1. 15.>
876,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률 시행령,"(3) Deleted. <May 25, 2021>",③ 삭제 <2021. 5. 25.>
931,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률 시행령,"(2) Deleted. <May 25, 2021>",② 삭제 <2021. 5. 25.>
...,...,...,...,...
365207,외무,재외공관 공증법 시행령,(3) Deleted. <by Presidential Decree No. 27939...,③ 삭제 <2017.3.20.>
365283,외무,재외동포재단법,"5. Deleted; <May 26, 2020>",5. 삭제 <2020. 5. 26.>
365284,외무,재외동포재단법,"6. Deleted; <May 26, 2020>",6. 삭제 <2020. 5. 26.>
365305,외무,재외동포재단법,"4. Deleted; <Jul. 16, 2013>",4. 삭제 <2013. 7. 16.>
